In [ ]:
%cd ..

In [ ]:
import os
import numpy as np
import imageio
import skimage.transform
import scipy.ndimage
import scipy.io
import matplotlib.pyplot as plt
import h5py

In [ ]:
x_slice = slice(1665, 3177)
y_slice = slice(987, 2499)
size_big = 256
down_factor = 8
levin_kernel_idx = 6
gaussian_kernel_idx = 3
output_folder = "figures/deg_models"


def resize_big(img):
    return skimage.transform.resize(
        img, (size_big, size_big), anti_aliasing=True, order=3
    )


def downsample(img):
    return img[::down_factor, ::down_factor, :]


def add_noise(img, noise_sigma):
    rng = np.random.default_rng()
    noise = rng.normal(0, noise_sigma, img.shape)
    return np.clip(img + noise, 0, 1)


def blur(img, kernel):
    res = []
    for c in range(3):
        res.append(scipy.ndimage.convolve(img[..., c], kernel))
    return np.stack(res, axis=-1)


def show_img(img):
    plt.imshow(img)
    plt.show()


def save_img(img, filename):
    if img.shape[0] != size_big:
        img = skimage.transform.resize(img, (size_big, size_big), order=0)
    imageio.imsave(os.path.join(output_folder, filename), img)


def save_normalized(x, filename):
    save_img(x / x.max(), filename)


In [ ]:
# Bear
orig = imageio.imread("notebooks/bear.jpg")
show_img(orig)

# Mask
mask = (imageio.imread("notebooks/mask.png") == 255).astype(np.float32)
show_img(mask)

# Levin blur kernel
with h5py.File("eval_utils/kernels/Levin09.mat", "r") as f:
    levin_kernel = f[f["kernels"][()][levin_kernel_idx][0]][()].astype("float32")
show_img(levin_kernel)

# Gaussian blur kernel
gaussian_kernel = scipy.io.loadmat("eval_utils/kernels/kernels_12.mat")["kernels"][0][
    gaussian_kernel_idx
][5:20,5:20]
show_img(gaussian_kernel)

# Denoising

In [ ]:
x = resize_big(orig[y_slice, x_slice])
save_img(x, "gt.png")

# Noise
x = add_noise(x, 0.2)
save_img(x, "denoising_01.png")
show_img(x)

# Deblurring

In [ ]:
x = resize_big(orig[y_slice, x_slice])

# Blur
x = blur(x, levin_kernel)
save_img(x, "deblurring_01.png")
show_img(x)

# Kernel
save_normalized(levin_kernel, "deblurring_kernel.png")

# Noise
x = add_noise(x, 0.15)
save_img(x, "deblurring_02.png")
show_img(x)

# Single Image Super-Resolution

In [ ]:
x = resize_big(orig[y_slice, x_slice])

# Blur
x = blur(x, gaussian_kernel)
save_img(x, "sisr_01.png")
show_img(x)

# Kernel
save_normalized(gaussian_kernel, "sisr_kernel.png")

# Downsample
x = downsample(x)
save_img(x, "sisr_02.png")
show_img(x)

# Noise
x = add_noise(x, 0.05)
save_img(x, "sisr_03.png")
show_img(x)

# Multi Frame Super-Resolution

In [ ]:
slices = [
    (1123, 1801, 1136),
    (731, 1361, 1672),
    (723, 1593, 2096),
    (1051, 1897, 1408),
]

for idx, (y, x, w) in enumerate(slices):
    
    # Warp
    ys = slice(y, y+w)
    xs = slice(x, x+w)
    x = resize_big(orig[ys, xs])
    save_img(x, f"mfsr_01_{idx}.png")

    # Blur
    x = blur(x, gaussian_kernel)
    save_img(x, f"mfsr_02_{idx}.png")

    # Downsample
    x = downsample(x)
    save_img(x, f"mfsr_03_{idx}.png")

    # Noise
    x = add_noise(x, 0.05)
    save_img(x, f"mfsr_04_{idx}.png")
    show_img(x)

# Kernel
save_normalized(gaussian_kernel, "mfsr_kernel.png")

# Inpainting

In [ ]:
x = resize_big(orig[y_slice, x_slice])

# Apply mask
x = (1 -mask) * x + mask
save_img(x, "inpainting_01.png")
show_img(x)

# Save mask
save_img(mask, "inpainting_mask.png")

# Noise
x = add_noise(x, 0.1)
save_img(x, "inpainting_02.png")
show_img(x)

# Noise

In [ ]:
x = np.ones((size_big, size_big, 3)) * 0.5
x = add_noise(x, 10)
save_img(x, "noise.png")
show_img(x)

# Machine Learning Model

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_datasets_bw as tfdsbw

images = (
    tfds.load("cbsd68", split="test")
    .map(tfdsbw.get_image)
    .map(tfdsbw.to_float32)
    .map(tfdsbw.from_255_to_1_range)
)

num_images = 4

for idx, img in enumerate(images):
    x = img.numpy()
    h, w, _ = x.shape

    # Make square
    if w > h:
        border = (w - h) // 2
        x = x[:, border:(border + h)]
    elif w < h:
        border = (h - w) // 2
        x = x[border:(border + w), :]

    x = resize_big(x)
    show_img(x)
    save_img(x, f"ml_{idx:02d}_clear.png")

    # Degrade
    x = blur(x, levin_kernel)
    x = add_noise(x, 0.15)
    show_img(x)
    save_img(x, f"ml_{idx:02d}_blurry.png")

    if idx + 1 >= num_images:
        break


In [ ]:
x = resize_big(orig[y_slice, x_slice])

# Blur
x = blur(x, gaussian_kernel)
x = add_noise(x, 0.15)
save_img(x, "ml_other_blurry.png")
show_img(x)